In [1]:
import argparse, csv
from tensorflow import keras
from tensorflow.python.keras.models import model_from_json
from keras.utils import to_categorical
import pandas as pd

from tools.metadata_tools import *

Using TensorFlow backend.


In [2]:
test = '../Results/json_detection'
train = '../Results/json_detection/train'
model = 'models/trainingJSON_2020-06-16_12:32:03'

json = open(model+'.json', 'r')
model_json = json.read()
json.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights(model+'.h5')

matrix_metadata = metadata_to_matrix(test, "json")
test_data = matrix_metadata[:,:14].astype(np.float32)
test_label = to_categorical(matrix_metadata[:,14].astype(np.float32).astype(np.int8))

matrix_metadata = metadata_to_matrix(train, "json")
train_data = matrix_metadata[:,:14].astype(np.float32)
train_label = to_categorical(matrix_metadata[:,14].astype(np.float32).astype(np.int8))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
from monumai.monument import Monument
names = [el for sublist in list(Monument.ELEMENT_DIC.values()) for el in sublist]

In [18]:
from anchor import utils
from anchor import anchor_tabular

target = 3
name = ['not ' + Monument.STYLES_HOTONE_ENCODE[target], Monument.STYLES_HOTONE_ENCODE[target]]

explainer = anchor_tabular.AnchorTabularExplainer(
    Monument.STYLES_HOTONE_ENCODE,
    names,
    train_data)

def predict_fn(model, target):
    def pred(el):
        preds = model.predict(el.reshape(-1,14))
        return np.argmax(preds,axis=1) == target
    return pred

In [19]:
idx = 120
np.random.seed(1)
print('Prediction: ', explainer.class_names[np.argmax(loaded_model.predict(train_data[idx].reshape(1,-1)))])
exp = explainer.explain_instance(train_data[idx], predict_fn(loaded_model,target), threshold=0.1)

Prediction:  R


In [20]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor: fronton-partido <= 0.00
Precision: 0.92
Coverage: 0.61


In [6]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular

In [7]:
# make sure you have adult/adult.data inside dataset_folder
dataset_folder = './Data/'
dataset = utils.load_dataset('adult', balance=True, dataset_folder=dataset_folder, discretize=True)

In [8]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
c.fit(dataset.train, dataset.labels_train)
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, c.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, c.predict(dataset.test)))

Train 0.9350338780390594
Test 0.8489483747609943


In [9]:
explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)

In [10]:
idx = 0
np.random.seed(1)
print('Prediction: ', explainer.class_names[c.predict(dataset.test[idx].reshape(1, -1))[0]])
exp = explainer.explain_instance(dataset.test[idx], c.predict, threshold=0.95)

Prediction:  b'>50K'


In [11]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor: Education = Bachelors AND Relationship = Husband AND Occupation = Sales
Precision: 0.97
Coverage: 0.02


In [12]:
c.predict(dataset.test[idx].reshape(1, -1))[0]

1